【メモ】[Enhancing Short-Term Mean-Reversion Strategies](https://www.quantopian.com/posts/enhancing-short-term-mean-reversion-strategies-1?utm_source=forum-content&utm_medium=twitter&utm_campaign=enhancing-short-term-mean-reversion-strategies-1)


In [ ]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor, Latest, SimpleMovingAverage, Returns, AnnualizedVolatility
from quantopian.pipeline.filters import Q500US
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.morningstar import Sector 
from quantopian.pipeline.data.eventvestor import EarningsCalendar
from quantopian.pipeline.factors.eventvestor import (
BusinessDaysUntilNextEarnings,
BusinessDaysSincePreviousEarnings
)

import pandas as pd 
import numpy as np

+ FairValueでLow Volの場合Short Term Mean Reversionが効くようだ
+ Fair Value = EV/EBITDARatioが０に近い．
+ 0以上どこかの大企業（AAPLとか）くらい，みたいなFilterをかけるといいかもしれない


In [ ]:
pipe = Pipeline()
pipe.set_screen(Q500US())

pipe.add(Latest(inputs=[USEquityPricing.close]), "Latest") ## pipe.add(USEquityPricing.close.latest, 'close') 
pipe.add(Returns(window_length=2), "Return") ## 前日クローズから本日のクローズまで
pipe.add(Returns(window_length=6), "5dayReturn") ## 前日クローズから本日のクローズまで
#pipe.add(Returns(window_length=3) - Returns(window_length=2), "Return as of 2days before") ## 2日前クローズから前日のクローズまで
pipe.add(AnnualizedVolatility(window_length=6), "Vol5day")

# BusinessDaysSincePreviousEarningsは，前回の決算日から何日過ぎているかをIntで返す．1であれば前日が決算日だったということ
# 無料じゃなかった
# pipe.add(EarningsCalendar.previous_announcement.latest, 'previous_announcement')
# pipe.add(EarningsCalendar.next_announcement.latest, 'next_announcement')
# pipe.add(BusinessDaysSincePreviousEarnings(), "business_days_since")


    

pipe.add(Sector(), "Sector")

pipe.add(morningstar.valuation_ratios.ev_to_ebitda.latest, "EV/EBITDA ratio")

results = run_pipeline(pipe, start_date="2016-11-1", end_date="2017-11-15")

In [ ]:
idx = pd.IndexSlice
results.loc[idx[:,[symbols("AAPL")]],:]



[野村證券 | EV/EBITDA倍率（証券用語解説集）](http://www.nomura.co.jp/terms/japan/i/ev_ebitda.html)
> EV（企業価値）がEBITDAの何倍になっているかを表す指標で、企業の買収に必要な時価総額と、買収後の純負債の返済に必要な金額を、EBITDAの何年分で賄えるかを表す。簡易買収倍率とも呼ばれている。
企業活動において、電機・機械、自動車などの産業は、世界規模で企業活動をおこなっているため、株価を比較する際に共通の尺度が必要となる。現在、世界的な株価比較の尺度として、EV/EBITDA倍率が広く使われている。

[野村證券 | EV（証券用語解説集）](http://www.nomura.co.jp/terms/english/e/ev.html)
> Enterprise Valueの略称で和訳は企業価値。会社が生み出す将来のフリーキャッシュフローを割引いた現在価値のことをいう。

In [ ]:
results[(results["EV/EBITDA ratio"] < 20) & (results["EV/EBITDA ratio"] > 0)].ix["2017-11-1"].sort_values(by="Vol5day")["EV/EBITDA ratio"].plot(kind="hist", alpha=.5)

In [ ]:
results[(results["EV/EBITDA ratio"] < 10) & (results["EV/EBITDA ratio"] > 0) & (results["Vol5day"] < 0.5)]["Vol5day"].plot(kind="hist")

In [ ]:
aapl = results.loc[idx[:,[symbols("AAPL")]],:]


In [ ]:
aapl[["Latest", "EV/EBITDA ratio"]].apply(np.log).plot(secondary_y = "Latest")

In [ ]:
symbols = results.ix["2017-11-1"].index
prices = get_pricing(symbols, start_date="2017-11-1", end_date="2017-11-8", fields="price")

In [ ]:
daily_rets=np.log(prices/prices.shift(1))


In [ ]:
rets=(prices.iloc[-2] - prices.iloc[0]) / prices.iloc[0]

In [ ]:
stdevs=daily_rets.std(axis=0)


In [ ]:
results[(results["EV/EBITDA ratio"] > 0 )  & (results["EV/EBITDA ratio"] < 10 ) & (results["Vol5day"] < 0.2)].ix["2017-11-2"].sort_values(by=["EV/EBITDA ratio", "Vol5day"], ascending=[True,True])

In [ ]:
get_pricing("ZION", start_date="2017-11-1", end_date="2017-11-8", fields="price").plot()